In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ds = pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')

In [ ]:
ds.head()

In [ ]:
ds.info()

In [ ]:
ds.isnull().sum()

In [ ]:
ds = ds.drop('Cabin',axis=1)

In [ ]:
ds.columns

In [ ]:
ds['Embarked'].value_counts()

## Port of Embarkation:C = Cherbourg, Q = Queenstown, S = Southampton

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
ds['Embarked'] = le.fit_transform(ds['Embarked'])

In [ ]:
ds['Embarked'].value_counts()

In [ ]:
ds['Sex'] = le.fit_transform(ds['Sex'])

In [ ]:
ds_num = ds.select_dtypes(include=['number'])

In [ ]:
ds = ds.drop(columns=['Name','Ticket'])

In [ ]:
ds.head()

In [ ]:
ds_num = ds_num.drop(['Survived'],axis=1)

In [ ]:
ds_num.columns

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
def imputing_reg(ds_num):
    ds_copy = ds_num.copy()
    
    for column in ds_copy.columns:
        if ds_copy[column].isnull().sum() > 0:
            known = ds_copy[column].notnull()
            unknown = ds_copy[column].isnull()
            
            xtrain = ds_copy[known].drop(column,axis=1)
            ytrain = ds_copy[known][column]
            
            model = HistGradientBoostingRegressor()
            model.fit(xtrain,ytrain)
            
            xtest = ds_copy[unknown].drop(column,axis=1)
            ypred = model.predict(xtest)
            
            ds_num.loc[unknown,column] = ypred
            
    return ds_num

ds_copy = imputing_reg(ds_num)
            
            
            

In [ ]:
ds_num.isnull().sum()

In [ ]:
ds['Age'] = ds_num['Age']

In [ ]:
ds.isnull().sum()

In [ ]:
ds.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ds.boxplot()
plt.xticks(rotation=45)
plt.title('outliers in the titanic dataset')
plt.xlabel('variables')
plt.ylabel('values')
plt.show()

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
numeric_data = ds.drop('Survived',axis=1)

In [ ]:
numeric_data = numeric_data.select_dtypes(include=['number'])

In [ ]:
numeric_data.columns

In [ ]:
mdl = IsolationForest(contamination=0.05,random_state=42)
mdl.fit(numeric_data)

## predict outliers in the data

In [ ]:
outliers = mdl.predict(numeric_data)

## convert prediction results to boolean array ( true for outliers, false for inliers )

In [ ]:
outliers = outliers == -1

## identify and print indices of outliers

In [ ]:
outlier_indices = numeric_data.index[outliers]
print("Outlier Indices:", list(outlier_indices))

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (16,8))
sns.scatterplot(x=numeric_data.iloc[:,0] , y=numeric_data.iloc[:,1], hue=outliers , palette = {True : 'r',False:'b'})
plt.title('outliers detection using isolation forest')
plt.show()

In [ ]:
ds.head()

In [ ]:
sns.countplot(x=ds['Survived'])

In [ ]:
plt.figure(figsize=(16,8))
sns.pairplot(ds,hue='Survived')

## by analizing the diagonal graphs we can see that they completely overlap so we can't use logistic regression to classify the dataset

In [ ]:
x = ds.drop('Survived',axis=1)
y = ds['Survived']

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest =train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(xtrain,ytrain)

In [ ]:
ypred = rfc.predict(xtest)

In [ ]:
ypred[:5]

In [ ]:
from sklearn.metrics import classification_report as cr
print(cr(ypred,ytest))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [ ]:
dtc.fit(xtrain,ytrain)

In [ ]:
ypred_dtc = dtc.predict(xtest)

In [ ]:
ypred_dtc[:5]

In [ ]:
print(cr(ytest,ypred_dtc))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv = cross_val_score(rfc,xtrain,ytrain,cv=15)
cv

In [ ]:
ypred_cv = rfc.predict(xtest)

In [ ]:
ypred_cv

In [ ]:
print(cr(ypred_cv,ytest))